In [1]:
import os
os.chdir("/Users/aliahmadi/Documents/Projects/RNA-Secondary-Structure-Prediction/notebook/EKH-25")

## Imports

In [2]:
from collections import defaultdict
import matplotlib.pyplot as plt
from scipy.linalg import expm
from grammar.pcnf import PCNF
from Bio import Phylo, SeqIO
from copy import deepcopy
from io import StringIO
import networkx as nx
from math import log 
import numpy as np
import shutil
import pickle
import random

___

## Global Variables

In [4]:
predicted_struct = {}

## Pairing Charecters

In [5]:
def get_pair_start(pair_end):
    pairing_chars = [("<", ">"), ("(", ")"), ("[", "]"), ("{", "}")]
    return [val for val in pairing_chars if val[1] == pair_end][0][0]

In [6]:
def is_pair_start(char):
    pairing_chars = [("<", ">"), ("(", ")"), ("[", "]"), ("{", "}")]
    return char in [p[0] for p in pairing_chars]

In [7]:
def is_pair_end(char):
    pairing_chars = [("<", ">"), ("(", ")"), ("[", "]"), ("{", "}")]
    return char in [p[1] for p in pairing_chars]

## Create Tree

In [13]:
def create_tree(input_sequences, filename = "./outputs/tree.nwk", draw=False):
  names = list(input_sequences.keys())
  sequences = list(input_sequences.values())
  
  if len(names) <= 2:
    return None
  
  os.mkdir("./tmp")

  if len(names) > 2:
    phylip_file = "./tmp/sequences.phylip"
    with open(phylip_file, "w") as f:
      f.write(f"{len(sequences)} {len(sequences[0])}\n\n")
      for i, seq in enumerate(sequences):
        f.write(f"{names[i]}\t{seq}\n") 
    
    # !./phyml -i tmp/sequences.phylip -m GTR
    !./phyml -i tmp/sequences.phylip -m GTR > /dev/null 2>&1

    
    output_tree = Phylo.read(phylip_file + "_phyml_tree.txt", 'newick')

    output_tree.root_at_midpoint()
  
  for index, clade in enumerate(output_tree.find_clades()):
    if not clade.name:
      clade.name = str(index)
  
  shutil.rmtree("./tmp")
  Phylo.write(output_tree, filename, "newick")
  if draw:
    Phylo.draw(output_tree)
  
  return output_tree

## Get Mutation Probablities

In [18]:
def get_mutation_probablities(mutation_rate_values, time, order_array = ["A", "C", "G", "U"], gappy=False):
    mutation_rate_matrix = np.zeros((len(order_array), len(order_array)), np.float64)
    
    for i_index, i_value in enumerate(order_array):
        for j_index, j_value in enumerate(order_array):
            mutation_rate_matrix[i_index,j_index] = mutation_rate_values[(i_value, j_value)]
    
    probability_rate_matrix = expm(mutation_rate_matrix * time)
    
    probability_rate_values = defaultdict(float)
    for i_index, i_value in enumerate(order_array):
        for j_index, j_value in enumerate(order_array):
            probability_rate_values[(i_value, j_value)] = probability_rate_matrix[i_index,j_index]
            
    if gappy:      
        if order_array == ["A", "C", "G", "U"]:
            for i_index, i_value in enumerate(order_array):
                for _, j_value in enumerate(["-"]):
                    probability_rate_values[(i_value, j_value)] = 1
        else:
            for i_index, i_value in enumerate(order_array):
                for j_index, j_value in enumerate(['A-', '-A', 'C-', '-C', 'G-', '-G', 'U-', '-U', '--']):
                    if j_value == "--":
                        probability_rate_values[(i_value, j_value)] = 1
                    else:
                        __probability = 0
                        for nocleotid in ["A", "C", "G", "U"]:
                            __j_value = j_value.replace('-', nocleotid)
                            __probability += probability_rate_values[(i_value, __j_value)]
                                
                        probability_rate_values[(i_value, j_value)] = __probability
    
        
    return probability_rate_values

# Check Column

In [20]:
def column_needed(tuple1, tuple2):
    return len(tuple1) == len(tuple2) and all(t1 == t2 for t1, t2 in zip(tuple1, tuple2))

In [21]:
def get_columns(input_sequences, leaf_order):
    columns = defaultdict(list)
    
    for i in range(len(list(input_sequences.values())[0])):
        column = tuple([
            input_sequences[name][i] for name in leaf_order
        ])
        columns[column].append(i)
    return columns

In [22]:
def check_column(input_sequences, column, leaf_order):
    __leaf_order = leaf_order[-len(column):]
    columns = get_columns(input_sequences, __leaf_order)
    
    if len(column[0]) == 1:
        for _column, _places in columns.items():
            if column_needed(column, _column):
                return True
    else:
        for _column1, _places1 in columns.items():
            left_column = tuple(
                    pair[0] for pair in column if pair
            )
            if column_needed(left_column, _column1):
                for _column2, _places2 in columns.items():
                    right_column = tuple(
                        pair[1] for pair in column if pair
                    )
                    if _places2 != [] and column_needed(right_column, _column2):
                        if _places1[0] < _places2[-1]:
                            return True
    
    return False       

## Post Order Traversal

In [23]:
def post_order_traversal(tree, current_node, interior_possible_values, tips_possible_values, rate_values, input_sequences, leaf_order = []): 
    current_columns_probability = defaultdict(lambda: defaultdict(float))
    for child in current_node.clades:
        # going to add a branch with a single nucleotide 
        if child.is_terminal():
            leaf_order.append(child.name)
            time = tree.distance(
                current_node.name, 
                child.name
            )
            mutation_probablities = get_mutation_probablities(rate_values, time, interior_possible_values, True)
            # have filled with some branch in last iteration
            if current_columns_probability:
                perv_columns_probability = deepcopy(current_columns_probability)
                current_columns_probability.clear()
                for left_column in perv_columns_probability:
                    for child_value in tips_possible_values:
                        if check_column(input_sequences, left_column + (child_value ,), leaf_order):
                            for root in perv_columns_probability[left_column]:
                                current_columns_probability[left_column + (child_value ,)][root] = (
                                    mutation_probablities[(root, child_value)] * 
                                    perv_columns_probability[left_column][root]
                                )
                perv_columns_probability.clear()
            
            # haven't filled with any branch in last iteration
            else:
                # for mutation in mutation_probablities:
                for child_value in tips_possible_values:
                    if check_column(input_sequences, (child_value ,), leaf_order):
                        for parent_value in interior_possible_values:
                            current_columns_probability[(child_value ,)][parent_value] = mutation_probablities[(parent_value, child_value)]
        # going to add a branch with a more than one nucleotide 
        else:    
            inner_columns_probability, _ = post_order_traversal(tree, child, interior_possible_values, tips_possible_values, rate_values, input_sequences, leaf_order)
            time = tree.distance(
                current_node.name, 
                child.name
            )
            mutation_probablities = get_mutation_probablities(rate_values, time, interior_possible_values)
            
            # have filled with some branch in last iteration
            if current_columns_probability:
                perv_columns_probability = deepcopy(current_columns_probability)
                current_columns_probability.clear()
                
                for left_column in perv_columns_probability:
                    for right_column in inner_columns_probability:
                        if check_column(input_sequences, left_column + right_column, leaf_order):
                            for root in perv_columns_probability[left_column]:
                                current_columns_probability[left_column + right_column][root] = sum(
                                    (
                                        mutation_probablities[(root, inner_root)] * 
                                        inner_columns_probability[right_column][inner_root] * 
                                        perv_columns_probability[left_column][root]
                                    ) for inner_root in inner_columns_probability[right_column])
                
                perv_columns_probability.clear()

            # haven't filled with some branch in last iteration
            else:
                for root_value in interior_possible_values:
                    for column in inner_columns_probability:
                        current_columns_probability[column][root_value] = sum(
                            (
                                mutation_probablities[(root_value, inner_root)] * 
                                inner_columns_probability[column][inner_root]
                            ) for inner_root in inner_columns_probability[column]
                        )
                    
    return current_columns_probability, leaf_order

## Get Columns Probability

In [24]:
def get_columns_probability(
    tree,  
    single_frequencies, 
    paired_frequencies, 
    single_rate_values, 
    paired_rate_values,
    input_sequences,
):
    single_interior_possible_values = ['A', 'C', 'G', 'U']
    paired_interior_possible_values = [c1 + c2 for c1 in single_interior_possible_values for c2 in single_interior_possible_values]
    
    single_tips_possible_values = ['A', 'C', 'G', 'U', '-']
    paired_tips_possible_values = [c1 + c2 for c1 in single_tips_possible_values for c2 in single_tips_possible_values]
    
    __single_columns_probability, leaf_order = post_order_traversal(
        tree, 
        tree.root, 
        single_interior_possible_values,
        single_tips_possible_values, 
        single_rate_values,
        input_sequences,
        []
    )
    single_columns_probability = defaultdict(float)
    for column in __single_columns_probability:
        single_columns_probability[column] = sum(
            __single_columns_probability[column][root] 
            * single_frequencies[root] for root in single_interior_possible_values
        )
    __single_columns_probability.clear()
    
    
    __paired_columns_probability, _ = post_order_traversal(
        tree, 
        tree.root, 
        paired_interior_possible_values, 
        paired_tips_possible_values,
        paired_rate_values,
        input_sequences,
        [],
    )
    paired_columns_probability = defaultdict(float)
    for column in __paired_columns_probability:
        paired_columns_probability[column] = sum(
            __paired_columns_probability[column][root] 
            * paired_frequencies[root] for root in paired_interior_possible_values
        )
    __paired_columns_probability.clear()


    return single_columns_probability, paired_columns_probability, leaf_order

## Save PCFG

In [25]:
def save_pcfg(pcfg, filename):
    unary_rules = pcfg.grammar.unary_rules
    binary_rules = pcfg.grammar.binary_rules
    
    for A, B, C in binary_rules:
        print(f"{A} -> {B} {C} {pcfg.q[(A, B, C)]}")
            
    for A, w in unary_rules:
        print(f"{A} -> {w} {pcfg.q[(A, w)]}")
    
    with open(f"{filename}.pcfg", "w+") as pcfg_file:
        for A, B, C in binary_rules:
            pcfg_file.write(f"{A} -> {B} {C} {pcfg.q[(A, B, C)]}\n")
            
        for A, w in unary_rules:
            pcfg_file.write(f"{A} -> {w} {pcfg.q[(A, w)]}\n")

## Extend Grammar

In [26]:
def extend_grammar(
    columns,
    pcfg,
    single_column_probs, 
    paired_column_probs,
    filename="Extended"
):
    unary_rules = pcfg.grammar.unary_rules
    binary_rules = pcfg.grammar.binary_rules
    
    with (open(f"{filename}.pcfg", "w+") as pcfg_file, open(f"{filename}.cfg", "w+") as cfg_file):
        for A, B, C in binary_rules:
            if A == "$M" and B == "B" and C == "F":
                for column, prob in single_column_probs.items():
                    if columns.get(column, False):
                        term = str(column).replace(" ", "")
                        pcfg_file.write(f"{A+term} -> B{term.lower()} F {pcfg.q[(A, B, C)]}" + "\n")
                        cfg_file.write(f"{A+term} -> B{term.lower()} F" + "\n")
            elif B == "$M" and C == "E":
                for column, prob in paired_column_probs.items():
                    left_column = tuple(
                        pair[0] for pair in column if pair
                    )
                    right_column = tuple(
                        pair[1] for pair in column if pair
                    )
                    if columns.get(left_column, False) and columns.get(right_column, False):
                        left_term = str(left_column).replace(" ", "")
                        right_term = str(right_column).replace(" ", "")
                        pcfg_file.write(f"{A} -> $M{left_term} E{right_term.lower()} {pcfg.q[(A, B, C)] * prob}" + "\n")
                        cfg_file.write(f"{A} -> $M{left_term} E{right_term.lower()}" + "\n")
            else:
                pcfg_file.write(f"{A} -> {B} {C} {pcfg.q[(A, B, C)]}" + "\n")
                cfg_file.write(f"{A} -> {B} {C}" + "\n")

        for A, w in unary_rules:
            if w == "s":
                for column, prob in single_column_probs.items():
                    if columns.get(column, False):
                        term = str(column).replace(" ", "")
                        pcfg_file.write(f"{A} -> {term} {pcfg.q[(A, w)] * prob}" + "\n")
                        cfg_file.write(f"{A} -> {term}" + "\n")
            elif w == "d":
                for column, prob in single_column_probs.items():
                    if columns.get(column, False):      
                        term = str(column).replace(" ", "")
                        pcfg_file.write(f"{A+term.lower()} -> {term} {pcfg.q[(A, w)]}" + "\n")
                        cfg_file.write(f"{A+term.lower()} -> {term}" + "\n")
            else:
                pcfg_file.write(f"{A} -> {w} {pcfg.q[(A, w)]}" + "\n")
                cfg_file.write(f"{A} -> {w}" + "\n")

## Generate Parse Tree

In [27]:
def gen_parse_tree(tree, table, start, end, non_terminal = "S", firstRun = True, layer=1):
    if firstRun:
        tree.add_node((start, end, "S"), layer=layer)
        layer += 1
        
    if start == end and table[(start, end, non_terminal)]:
        if non_terminal.startswith("B"):
            predicted_struct[start] = "("
        elif non_terminal.startswith("E"):
            predicted_struct[start] = ")"
        else:
            predicted_struct[start] = "."

    for _start, _end, _non_terminal in table[(start, end, non_terminal)]:
        tree.add_node((_start, _end, _non_terminal), layer=layer)
        new_layer = layer + 1
        tree.add_edge((_start, _end, _non_terminal), (start, end, non_terminal))
        gen_parse_tree(tree, table, _start, _end, _non_terminal, firstRun=False, layer=new_layer)

## Draw Parse Tree

In [28]:
def draw_parse_tree(table, start_point, end_point, show=False):
    parse_tree = nx.Graph()
    
    gen_parse_tree(parse_tree, table, start_point, end_point)
    
    if show:
        pos = nx.multipartite_layout(parse_tree, subset_key ="layer")
        nx.draw(
            parse_tree, 
            pos, 
            # with_labels=True, 
            node_color='#74b9ff', 
            node_size=50, 
            font_size=10
        )
        nx.draw_networkx_nodes(
            parse_tree, 
            pos, 
            nodelist=[(start_point, end_point, "S")], 
            node_color='#0984e3', 
            node_size=50
        )

## Get Total Sequence

In [29]:
def get_total_sequence(input_sequences, leaf_order):
    total_sequence = ""
    columns = defaultdict(bool)
    for i in range(len(list(input_sequences.values())[0])):
        column = tuple([
            input_sequences[name][i] for name in leaf_order
        ])
        if column.count("<") == len(leaf_order):
            total_sequence += "< "
        elif column.count(">") == len(leaf_order):
            total_sequence += "> "
        else:
            columns[column] = True
            total_sequence += str(column).replace(" ", "") + " "
    return total_sequence, columns

___

In [31]:
# Load from the file
with open("./primaries/parameters/_combined/frequencies.pkl", "rb") as file:
    (single_frequencies, 
     paired_frequencies, 
     singles_prob, 
     paireds_prob) = pickle.load(file)
    
with open("./primaries/parameters/_combined/mutation_rate.pkl", "rb") as file:
    (single_rate_values,
     paired_rate_values) = pickle.load(file)
    
pcfg = PCNF("./primaries/structure.cfg", "./primaries/parameters/_combined/structure.pcfg")

_combined_params = (single_frequencies, paired_frequencies, single_rate_values, paired_rate_values, pcfg)

___

## Input Sequences

In [34]:
validation = {
    "RF01862": [
        {
            "Seq1": "ACGCCUUUGUCUAACACCCCGCACCGCGAGCACUAUUUCCCGGCGGGGUGAUUUCAGAGGGCGGAGAUUC",
            "Seq2": "CCGUCUCUGUCUAACGCCUCACA-UGU---GCAGAAAUCGUUGUGGGGCGAUUUCAGGAGGCGGAGAUAU",
            "Seq3": "ACGUCUCUGUCUAACGGGGUGCGGUGC---UGCUCUGUCGGCGCACCUCGAUUUCAGGAGACGGAGAUGA",
            "Seq4": "ACGUCUCUGUCUAACGGGGUGCGACGC---UGCUCUGUCGGCGCACCUCGAUUUCAGGAGAUGGAGAUGA",
        },
                    ".(((((((......(((((((((..................)))))))))......)))))))......."
    ],
    
    "RF01722": [
        {
            "Seq1": "ACUGCCGGGACUACGCCGGGCAAGGCCGGC-GCCGU-GCCGCGCUGUGACCCCGGCGGGGCGCCU",
            "Seq2": "ACUGCCGGGACUACGCCGGAUAAGAGCGGC-UAUAU-GCCGCGCUGUGAAUCCGGCGGGGUUUUA",
            "Seq3": "AAAUUCUAGAU--GGGCUACAGAGAGCCGC-UUAC--GCGGCACUGUGAUGUAGCCUGACGGUGU",
            "Seq4": "ACG-CCGCGAC--GGGCUGUCGAGAGCCGC-GUCU--GCGGCGCUGUGAGACGGCCUGACGGCGU",
            "Seq5": "UGUGCCGGGACUACGCCGGGUGAGAGCGGC-GUGU--GCCGCCCUGUGAAUCCGGCGGGGUGCCU",
            "Seq6": "ACA-CAGCGAC--AGGCUGUUGAGAGCCGCCUCAGAGGCGGCGCUGUGAGACGGCCUGACGGUGU",
            "Seq7": "AGCGCCGGGACUACGCCGGGUGAGAGCGGC-UGGC--GCCGCACUGUGGGCCCGGCGGGGUGCCU",
            "Seq8": "ACAACCGCGAC--GGGCUGUGGAGAGCCGC-GCCC--GCGGCGCCGUGAAACAGCCUGACGGUGU",
        },
                    "..........(..((((((((....(((((.......))))).......))))))))........"
    ],
    
    "RF03537": [
        {
            "Seq01": "CCCCUGCAUCAUGAUAAGGC-CGAACAUGGUGCAUGAAAGGGGAGG",
            "Seq02": "CCCCCGCACCAUGACAAGGC-CGAACAUGGUGCACCAAAGGGGAGG",
            "Seq03": "CCCCCGCCCCAUGACAAGGC-CGAACAUGGAGCAUUAAAGGG-AGG",
            "Seq04": "CCCUUGCGUCCAGAGAAGGC-CGAACUGGGCGU---UAUAAGGAGG",
            "Seq05": "CCCCCGCACCAUG-GAAGGC-CAAACAUGGUGCAUG-AAGGGAAAG",
            "Seq06": "CAAAACCUCCCAGAGAAGGC-CGAACUGGGAGGCC-----AUGAAG",
            "Seq07": "CAAAGCCUCCCAGAGAGGGC-CGAACUGGGAGGUU-----AUGAAG",
            "Seq08": "CAAAAGGCCUCCUGGAAGGCUCACCAGGAGUUAGGCCAUUCUAGAG",
            "Seq09": "CAAAAGGCCUCCUGGAAGGCUCACCAGGAGUUAGGCCGUUU--AGG",
            "Seq10": "UGA---CCAUCCCUCAAGGCCGAGUGGGAUGCG------UAUGAAG",
        },
                     "((((.((((((((............))))))))......))))..."
    ],
    
    "RF01737":[
        {
            "Seq1": "UCAUGUGACGAACAACCCGAAGGCUAAGGCCAGGGGA-GUUCUGAUGA",
            "Seq2": "UCAUAUGACGAGCAACCCGAAGGUUUAGACCAGGGAA-GUUCUGAUGA",
            "Seq3": "UCAGGUGACAAACGACCCGAAGGUAGAUACCAGGGGA-GUUUUGAUGA",
            "Seq4": "CCGGAUGAUGGCCCGGGGGAACCCUAACGGGACCCCG-GGCCGGACGG",
            "Seq5": "ACAGAUGAUGCACUAUCCGAAGGCUUA-GCCAGGGUAUGUGUUGAUGU",
            "Seq6": "UCAUAUGACGAACAACCCGAAGGUUAAAACCAGGGAA-GUUCUGAUGA",
            "Seq7": "UCAUAUGACGAGCAACCCGAAGGCUAAAGCCAGGGAA-GUUCUGAUGA",
            "Seq8": "CCAGAUGAGGCACCACUCGAAGGC-AAUGCCAAAGUG-GUGCUGAUGG",
        },
                    "((((.....((((..(((...(((....))).)))...))))..))))"
    ]
}

___

In [35]:
def predict_structure(input_sequences, single_frequencies, paired_frequencies, 
                      single_rate_values, paired_rate_values, pcfg, first_start_ratio, 
                      second_start_ratio, first_accelerat_ratio, second_accelerat_ratio, flag_ratio
                     ):    
    # Step 1: Create the initial tree
    estimated_tree = create_tree(input_sequences)

    # Step 2: Calculate single and paired column probabilities
    single_columns_probability, paired_columns_probability, leaf_order = get_columns_probability(
        estimated_tree,
        single_frequencies,
        paired_frequencies,
        single_rate_values,
        paired_rate_values,
        input_sequences,
    )

    # Step 3: Get the total sequence and columns based on leaf order
    total_sequence, columns = get_total_sequence(input_sequences, leaf_order)

    # Step 4: Extend the grammar
    extend_grammar(
        columns,
        pcfg,
        single_columns_probability,
        paired_columns_probability,
        filename="./outputs/Extended"
    )

    # Step 5: Read from extended grammar file and run CYK algorithm
    extended_pcfg = PCNF("./outputs/Extended.cfg", "./outputs/Extended.pcfg")

    log_prob, table = extended_pcfg.sentence_prob(total_sequence, first_start_ratio, first_accelerat_ratio)
    # Step 6: Parse table to draw tree and generate structure
    global predicted_struct
    predicted_struct = {}
    draw_parse_tree(table, 1, len(list(input_sequences.values())[0]))
    structure = "".join(predicted_struct.values())

    return structure, log_prob

In [37]:
def parse_dot_bracket(dot_bracket):
    stack = {}
    pairs = []
    for i, char in enumerate(dot_bracket):
        if char in "([{<":
            stack[char] = stack.get(char, []) + [i]
        elif char in ")]}>":
            if char in "([{<)]}>":  # Ensure char is a valid closing bracket
                match_index = "([{<)]}>".index(char)
                match = "([{<"[match_index - 4]  # Correct matching index logic
                if stack.get(match):
                    j = stack[match].pop()
                    pairs.append((j, i))
    return sorted(pairs)

def check_structure(predicted_structure, original_structure):
    predicted_pairs = set(parse_dot_bracket(predicted_structure))
    original_pairs = set(parse_dot_bracket(original_structure))

    true_positives_pairs = len(predicted_pairs & original_pairs)
    false_positives_pairs = len(predicted_pairs - original_pairs)
    false_negatives_pairs = len(original_pairs - predicted_pairs)

    total_positions = len(predicted_structure)
    predicted_unpaired = set(i for i in range(total_positions) if all(i not in pair for pair in predicted_pairs))
    original_unpaired  = set(i for i in range(total_positions) if all(i not in pair for pair in original_pairs))
    
    predicted_unpaired = set(i for i in range(total_positions) if all(i not in pair for pair in predicted_pairs))
    original_unpaired  = set(i for i in range(total_positions) if all(i not in pair for pair in original_pairs))
    
    true_positives_unpairs = len(predicted_unpaired & original_unpaired)
    false_positives_unpairs = len(predicted_unpaired - original_unpaired)
    false_negatives_unpairs= len(original_unpaired - predicted_unpaired)
    
    # Calculate weights for paired and unpaired
    paired_weight = len(original_pairs) * 2
    unpaired_weight = len(original_unpaired)

    return (true_positives_pairs, 
            false_positives_pairs, 
            false_negatives_pairs, 
            true_positives_unpairs, 
            false_positives_unpairs, 
            false_negatives_unpairs, 
            paired_weight, unpaired_weight)


def calc_f1_score(true_positives, false_positives, false_negatives):
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) else 0
    f1_score = (2 * precision * recall) / (precision + recall) if precision + recall > 0 else 0
    return f1_score

def calc_weighted_mean_f1_score(first_weight, first_f1, second_weight, second_f1):
    return ((first_weight * first_f1 + second_weight * second_f1) / (first_weight + second_weight))


In [38]:

# Define the evaluation function
def evaluate_individual(individual, test_data):
    _true_positives_pairs, _false_positives_pairs, _false_negatives_pairs = 0, 0, 0
    _true_positives_unpairs, _false_positives_unpairs, _false_negatives_unpairs = 0, 0, 0
    _paired_weight, _unpaired_weight = 0, 0

    for test_name, (alignment, actual_structure) in test_data.items():
        predicted_structure, _ = predict_structure(alignment, *_combined_params, *individual)
        
        (true_positives_pairs, 
        false_positives_pairs, 
        false_negatives_pairs, 
        true_positives_unpairs, 
        false_positives_unpairs, 
        false_negatives_unpairs, 
        paired_weight, unpaired_weight) = check_structure(predicted_structure, actual_structure)

        _true_positives_pairs += true_positives_pairs
        _false_positives_pairs += false_positives_pairs
        _false_negatives_pairs += false_negatives_pairs
        _true_positives_unpairs += true_positives_unpairs
        _false_positives_unpairs += false_positives_unpairs
        _false_negatives_unpairs += false_negatives_unpairs
        _paired_weight += paired_weight
        _unpaired_weight += unpaired_weight

    paired_f1_score = calc_f1_score(_true_positives_pairs, _false_positives_pairs, _false_negatives_pairs)
    unpaired_f1_score = calc_f1_score(_true_positives_unpairs, _false_positives_unpairs, _false_negatives_unpairs)

    mean_f1_score = calc_weighted_mean_f1_score(_unpaired_weight, unpaired_f1_score, _paired_weight, paired_f1_score)
    return mean_f1_score
